In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
games = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/games.csv")
player_play = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/player_play.csv")
players = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/players.csv")
plays = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/plays.csv")
tracking_week_1 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv")
#tracking_week_2 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv")
#tracking_week_3 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv")
#tracking_week_4 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv")
#tracking_week_5 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv")
#tracking_week_6 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv")
#tracking_week_7 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv")
#tracking_week_8 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv")
#tracking_week_9 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv")

In [ ]:
games.head()

In [ ]:
print(player_play.shape)
player_play.head()

In [ ]:
print(players.shape)
players.head()

In [ ]:
print(plays.shape)
plays.head()

In [ ]:
plays.sort_values(by=["gameId", "playId"])

In [ ]:
print(tracking_week_1.shape)
tracking_week_1.head()

In [ ]:
tracking_week_1.gameId.unique()

In [ ]:
tracking_week_1.frameType.unique()

In [ ]:
play1 = tracking_week_1[(tracking_week_1.gameId == 2022091200) & (tracking_week_1.playId == 64)]

In [ ]:
print(tracking_week_1.x.aggregate(["min", "max"]))
print(tracking_week_1.y.aggregate(["min", "max"]))

In [ ]:
before_snap = play1[play1.frameType == "BEFORE_SNAP"]
snap = play1[play1.frameType == "SNAP"]
after_snap = play1[play1.frameType == "AFTER_SNAP"]

In [ ]:
players = before_snap.displayName.unique()
players

In [ ]:
player0 = before_snap[before_snap.displayName == players[0]]
player0

In [ ]:
player0.loc[:, ["x", "y"]].isna().sum()

In [ ]:
player0.time.aggregate(["min", "max"])

In [ ]:
plt.xlim(0, 120)
plt.ylim(0, 53)
for i in range(len(before_snap.displayName.unique())):
    player = before_snap[before_snap.displayName == players[i]].reset_index(drop=True)
    if player.displayName[0] == "football":
        plt.scatter(player.x, player.y, c="red")
        plt.text(list(player.x)[-1], list(player.y)[-1], "FB")
    else:
        plt.scatter(player.x, player.y, c=player.frameId)
        #plt.text(list(player.x)[-1], list(player.y)[-1], list(player.jerseyNumber)[-1])
plt.title("before snap")
plt.show()

plt.xlim(0, 120)
plt.ylim(0, 53)
for i in range(len(snap.displayName.unique())):
    player = snap[snap.displayName == players[i]].reset_index(drop=True)
    if player.displayName[0] == "football":
        plt.scatter(player.x, player.y, c="red")
        plt.text(list(player.x)[-1], list(player.y)[-1], "FB")
    else:
        plt.scatter(player.x, player.y, c=player.frameId)
        #plt.text(list(player.x)[-1], list(player.y)[-1], list(player.jerseyNumber)[-1])
plt.title("during snap")
plt.show()


for i in range(len(after_snap.displayName.unique())):
    player = after_snap[after_snap.displayName == players[i]].reset_index(drop=True)
    if player.displayName[0] == "football":
        plt.scatter(player.x, player.y, c="red")
        plt.text(list(player.x)[-1], list(player.y)[-1], "FB")
    else:
        plt.scatter(player.x, player.y, c=player.frameId)
        #plt.text(list(player.x)[-1], list(player.y)[-1], list(player.jerseyNumber)[-1])
plt.title("after snap")
plt.show()

In [ ]:
# Player start vs end (done)
# Ball path vs end (nan make a different color) (almost done)
# Average distance moved by each player
# Figuring out a function to determine if a player is moving the ball
# How much each player moves the ball 

In [ ]:
def ball_distance_dictionary(player_df : pd.DataFrame) -> (dict, int): 
    player_sep = {}
    football = player_df[player_df.displayName == 'football'].reset_index(drop=True)
    for i in range(len(player_df.displayName.unique())):
        player = player_df[player_df.displayName == players[i]].reset_index(drop=True)
        if player.displayName[0] != "football":
            distances = list(zip(player.x, player.y))
            seps = []
            for idx, (x_dist, y_dist) in enumerate(distances):
                sep = ((football.x[idx] - x_dist)**2 + (football.y[idx] - y_dist)**2)
                seps.append(sep)
                frames = len(seps)
        player_sep[players[i]] = seps
    return player_sep, frames

In [ ]:
def player_with_ball() -> list:
    player_sep, frames = ball_distance_dictionary(after_snap)
    player_with_ball = [None] * frames
    
    for frame in range(frames):
        curr = None
        curr_dist = 17209
        for player in players:
            if player_sep[player][frame] < 1:
                curr = player
                curr_dist = player_sep[player][frame]
            if curr is not None and player_sep[player][frame] < curr_dist:
                curr = player
                curr_dist = player_sep[player][frame]
        if curr is not None:
            player_with_ball[frame] = (curr, curr_dist)
        else:
            player_with_ball[frame] = ('In air', None)

    return player_with_ball

In [ ]:
# Function predicts the movement of the ball for the next k frames using linear regression performed on the x and y data. 

def predict_next_k_frames(k: int) -> pd.DataFrame: 
    players = after_snap['displayName'].unique()
    new_after_snap = pd.DataFrame()

    for value in players:
        player_data = after_snap.loc[after_snap['displayName'] == value]
        x_data = [(row['frameId'], row['x']) for _, row in player_data.iterrows()]
        y_data = [(row['frameId'], row['y']) for _, row in player_data.iterrows()]

        X = np.array([point[0] for point in x_data]).reshape(-1, 1)  # Reshape to 2D array
        x_pred = np.array([point[1] for point in x_data])
        model_x = LinearRegression()
        model_x.fit(X, x_pred)
        max_frameid = y_data[-1][0]
        next_frame_ids = np.array([max_frameid + i for i in range(1, k + 1)]).reshape(-1, 1)
        predicted_x = model_x.predict(next_frame_ids)    
        
        Y = np.array([point[0] for point in y_data]).reshape(-1, 1)  # Reshape to 2D array
        y_pred = np.array([point[1] for point in y_data])
        model_y = LinearRegression()
        model_y.fit(Y, y_pred)
        max_frameid = y_data[-1][0]
        next_frame_ids = np.array([max_frameid + i for i in range(1, k + 1)]).reshape(-1, 1)
        predicted_y = model_y.predict(next_frame_ids)    
        

        new_rows = pd.DataFrame({
            'frameId': next_frame_ids.flatten(),
            'x': predicted_x.flatten(),
            'y': predicted_y.flatten()
        })

        for col in after_snap.columns:
            if col not in new_rows.columns:  
                new_rows[col] = after_snap[col].iloc[-1]

        updated_player_data = pd.concat([player_data, new_rows], ignore_index=True)
        new_after_snap = pd.concat([new_after_snap, updated_player_data], ignore_index=True)

    return new_after_snap